# LangChain 1.0+ Introduction - create_agent & Quick Start
"""
=====================================================================
LangChain 1.0+ Quick Start Guide
=====================================================================

This notebook demonstrates the modern LangChain 1.0+ approach using
the new `create_agent` function - the simplest way to build agents.

Key Concepts Covered:
--------------------
1. create_agent() - The easiest way to build tool-using agents
2. Tool definition with @tool decorator
3. Agent invocation and message handling
4. Understanding agent responses

What's New in LangChain 1.0+:
----------------------------
- Simplified create_agent() replaces complex graph setup
- Unified model initialization with init_chat_model()
- Built-in tool calling support
- Automatic message handling

Architecture:
-------------
    User Message
         │
         ▼
    ┌─────────────────┐
    │  create_agent   │  ← Simplified agent creation
    │  (model+tools)  │
    └─────────────────┘
         │
         ├──► Tool Calls (if needed)
         │         │
         │         ▼
         │    ┌──────────┐
         │    │  Tools   │
         │    └──────────┘
         │         │
         ◄─────────┘
         │
         ▼
    Final Response

Updated for LangChain 1.0+ (2025-2026)
"""

In [ ]:
# =============================================================================
# Environment Setup
# =============================================================================
# Load environment variables for API keys
# Required: OPENAI_API_KEY (or other provider keys)

import os
from dotenv import load_dotenv

load_dotenv()

# Set API keys from environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

print("✅ Environment configured")

In [ ]:
# =============================================================================
# create_agent() - The Simplest Way to Build Agents (LangChain 1.0+)
# =============================================================================
"""
create_agent() is the new unified way to create tool-using agents.
It handles all the complexity of:
- Tool binding
- Message management
- Tool execution loop
- Response formatting

Parameters:
-----------
- model: str or ChatModel - The LLM to use (e.g., "gpt-4o", "gpt-4.1")
- tools: List[Callable] - Functions decorated with @tool
- system_prompt: str - Instructions for the agent
- checkpointer: Optional - For conversation persistence
- middleware: Optional - For advanced features like summarization

Returns:
--------
An agent that can be invoked with messages
"""

from langchain.agents import create_agent
from langchain_core.tools import tool

# =============================================================================
# Step 1: Define Tools
# =============================================================================
# Tools are Python functions decorated with @tool
# The docstring becomes the tool description for the LLM

@tool
def get_weather(location: str) -> str:
    """
    Get the current weather for a given location.
    
    Args:
        location: The city or location to get weather for (e.g., "New York", "London")
    
    Returns:
        A string describing the current weather conditions
    """
    # In production, this would call a weather API
    return f"The weather in {location} is sunny with a high of 75°F."

@tool
def calculate(expression: str) -> str:
    """
    Evaluate a mathematical expression.
    
    Args:
        expression: A mathematical expression to evaluate (e.g., "2 + 2", "10 * 5")
    
    Returns:
        The result of the calculation
    """
    try:
        # Safe evaluation for basic math
        result = eval(expression, {"__builtins__": {}}, {})
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

# =============================================================================
# Step 2: Create the Agent
# =============================================================================
# create_agent combines model + tools + system prompt into a ready-to-use agent

agent = create_agent(
    model="gpt-4o-mini",              # Model to use (supports OpenAI, Anthropic, etc.)
    tools=[get_weather, calculate],    # List of tool functions
    system_prompt="You are a helpful assistant that can check weather and do calculations. "
                  "Use the tools when needed to answer questions accurately."
)

print("✅ Agent created with tools:", [t.name for t in [get_weather, calculate]])

In [ ]:
# =============================================================================
# Step 3: Invoke the Agent
# =============================================================================
"""
Agent invocation accepts messages in multiple formats:
1. Dict format: {"messages": [{"role": "user", "content": "..."}]}
2. String format: {"messages": "your question"}
3. HumanMessage format: {"messages": [HumanMessage(content="...")]}

The agent automatically:
- Processes the user message
- Decides if tools are needed
- Executes tools if required
- Returns a final response
"""

# Example 1: Weather query (will use get_weather tool)
print("=" * 60)
print("Example 1: Weather Query (Uses Tool)")
print("=" * 60)

result = agent.invoke({
    "messages": [{"role": "user", "content": "What is the weather in New York?"}]
})

# The result contains the full message history
print(f"\n📩 Messages in response: {len(result['messages'])}")

# Let's examine each message
for i, msg in enumerate(result['messages']):
    msg_type = type(msg).__name__
    content_preview = msg.content[:100] if msg.content else "[No content - tool call]"
    print(f"  {i+1}. {msg_type}: {content_preview}")

# Get the final answer
final_answer = result['messages'][-1].content
print(f"\n✅ Final Answer: {final_answer}")

In [ ]:
# =============================================================================
# Example 2: Query Without Tool Use
# =============================================================================
"""
When a question doesn't require tools, the agent responds directly.
Notice the agent intelligently decides NOT to use the get_weather tool
because "australia" is too broad - it needs a specific city.
"""

print("=" * 60)
print("Example 2: Broad Query (Agent Decides No Tool Needed)")
print("=" * 60)

result = agent.invoke({"messages": "What is the weather in australia?"})

# Shorter response - no tool calls
print(f"\n📩 Messages: {len(result['messages'])}")
print(f"\n✅ Response: {result['messages'][-1].content[:500]}...")

In [ ]:
# =============================================================================
# Example 3: Using the Calculator Tool
# =============================================================================
print("=" * 60)
print("Example 3: Math Calculation (Uses Calculator Tool)")
print("=" * 60)

result = agent.invoke({"messages": "What is 15 * 7 + 23?"})
print(f"\n✅ Response: {result['messages'][-1].content}")

# =============================================================================
# Example 4: Multi-Tool Query
# =============================================================================
print("\n" + "=" * 60)
print("Example 4: Combined Query")
print("=" * 60)

result = agent.invoke({
    "messages": "What's the weather in Tokyo? Also, what is 100 divided by 4?"
})
print(f"\n✅ Response: {result['messages'][-1].content}")

# =============================================================================
# Summary: create_agent() Key Points
# =============================================================================
"""
create_agent() - The Modern Way to Build Agents:
------------------------------------------------

✅ SIMPLEST APPROACH:
   agent = create_agent(model="gpt-4o-mini", tools=[...], system_prompt="...")

✅ AUTOMATIC FEATURES:
   - Tool calling and execution
   - Message history management
   - Multi-turn conversations
   - Error handling

✅ TOOL DEFINITION:
   @tool
   def my_tool(param: str) -> str:
       '''Tool description (shown to LLM)'''
       return result

✅ INVOCATION FORMATS:
   agent.invoke({"messages": "question"})
   agent.invoke({"messages": [{"role": "user", "content": "..."}]})

✅ RESPONSE STRUCTURE:
   result["messages"] -> List of all messages (Human, AI, Tool)
   result["messages"][-1].content -> Final answer

Next Steps:
-----------
- 2-modelIntegration.ipynb: Different model providers
- 3-tools.ipynb: Advanced tool patterns
- 4-messages.ipynb: Message types and formatting
- 5-structureoutput.ipynb: Structured outputs with Pydantic
- 6-middleware.ipynb: Summarization and advanced features
"""
print("\n" + "=" * 60)
print("📚 Summary: create_agent() is the simplest way to build agents!")
print("=" * 60)